# Riot API로 받지 못하는 정보 OP.GG에서 크롤링하기

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

### 각 라인의 탑 티어 챔피언 구하기

In [2]:
url = 'https://www.op.gg/champion/statistics'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [90]:
item_df = pd.read_csv('../00. data/items.csv', index_col = 0)
spell_df = pd.read_csv('../00. data/spell.csv', index_col = 0)

In [72]:
# 아이템키로 아이템 이름 불러오는 함수
def searchItem(itemKey):
    item = item_df['name'][item_df['name'][item_df['key'] == itemKey].index[0]]
    return item 

In [93]:
# 영어이름의 스펠을 한글로 바꿔주는 함수
def translation_spell(eng):
    spell = spell_df['name'][spell_df['name'][spell_df['eng'] == eng].index[0]]

In [3]:
# 태그 지우는 함수
def removeTags(strings):
    strings = re.sub('<.+?>', '', str(strings), 0).strip().replace("'",'')
    return strings

In [77]:
# OP.GG 아이템이미지의 키를 구하는 함수
def findItem(itemLink):
    items = re.findall(r'\d\d\d\d.png', str(itemLink))
    items = [item.strip('.png') for item in items]
    
    return items

In [4]:
lanes = ['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT']

In [5]:
for lane in lanes:
    top5 = soup.find(attrs={'class':f'tabItem champion-trend-tier-{lane}'}).select('.champion-index-table__name')[:10] # 10개만 가져오기
    print(f'{lane}의 최고티어 챔피언:',removeTags(top5))

TOP의 최고티어 챔피언: [Irelia, Camille, Fiora, Sett, Sylas, Jax, Wukong, Aatrox, Viego, Shen]
JUNGLE의 최고티어 챔피언: [Lee Sin, Xin Zhao, Elise, Karthus, Viego, Nocturne, Kindred, RekSai, Shaco, Fiddlesticks]
MID의 최고티어 챔피언: [Katarina, Talon, LeBlanc, Zed, Anivia, Sylas, Yasuo, Viego, Aurelion Sol, Galio]
ADC의 최고티어 챔피언: [Ezreal, Ziggs, Vayne, Samira, KogMaw, Ashe, Jinx, Swain, KaiSa, Jhin]
SUPPORT의 최고티어 챔피언: [Leona, Lulu, Thresh, Senna, Xerath, Maokai, Nautilus, Blitzcrank, Karma, Shaco]


### 원하는 챔피언의 기록 가져오기

In [6]:
# 샘플은 탑라인의 이렐리아
lane = 'top'
champion ='irelia'
url = f'https://www.op.gg/champion/{champion}/statistics/{lane}'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [7]:
win_rank = soup.select('.champion-stats-trend-rank')[0].find('b').text + soup.select('.champion-stats-trend-rank')[0].find('span').text
pick_rank = soup.select('.champion-stats-trend-rank')[1].find('b').text + soup.select('.champion-stats-trend-rank')[1].find('span').text
print('승률 순위:', win_rank, '픽률 순위:', pick_rank)

승률 순위: 20/55 픽률 순위: 1/55


### 원하는 챔피언의 추천 세팅 

In [89]:
for i in range(10):
    tmp = soup.select('td.champion-overview__data.champion-overview__border.champion-overview__border--first')[i].find_all('img')
    keys = findItem(tmp)
    items = [searchItem(int(key)) for key in keys]
    items = ', '.join(items)
    if i < 2:
        msg = '시작아이템 :'
    elif i >= 2 and i < 7:
        msg = '추천 빌드 :'
    else:
        msg = '신발 :'
    print(msg + items)
        

시작아이템 :도란의 검, 체력 물약
시작아이템 :부패 물약
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 구인수의 격노검
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 마법사의 최후
추천 빌드 :몰락한 왕의 검, 신성한 파괴자, 스테락의 도전
추천 빌드 :몰락한 왕의 검, 선혈포식자, 스테락의 도전
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 죽음의 무도
신발 :판금 장화
신발 :헤르메스의 발걸음
신발 :명석함의 아이오니아 장화


### 매치업

In [171]:
url = f'https://www.op.gg/champion/{champion}/statistics/{lane}/matchup'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [170]:
for i in range(30):
    champ = soup.select('.champion-matchup-list__champion')[i].find('span').text
    winRate = soup.select('.champion-matchup-list__champion')[i].find_all('span')[1].string
    winRate = [item.strip() for item in winRate if str(item)]
    winRate = ''.join(winRate)
    play_rate = soup.select('.champion-matchup-list__totalplayed')[i].find('span').text
    play_num = soup.select('.champion-matchup-list__totalplayed')[i].find('small').text
    print(f'{champion} vs {champ} 상대승률 :',winRate)
    print(f'통계량: {play_num} 통계 비율 {play_rate}')

irelia vs Sett 상대승률 : 49.91%
통계량: 13,177 통계 비율 7.94%
irelia vs Fiora 상대승률 : 49.18%
통계량: 10,612 통계 비율 6.39%
irelia vs Sylas 상대승률 : 51.3%
통계량: 9,532 통계 비율 5.74%
irelia vs Camille 상대승률 : 50.31%
통계량: 6,611 통계 비율 3.98%
irelia vs Renekton 상대승률 : 52.46%
통계량: 6,184 통계 비율 3.72%
irelia vs Malphite 상대승률 : 51.19%
통계량: 5,956 통계 비율 3.59%
irelia vs Jax 상대승률 : 48.89%
통계량: 5,835 통계 비율 3.51%
irelia vs Gwen 상대승률 : 54.13%
통계량: 5,792 통계 비율 3.49%
irelia vs Aatrox 상대승률 : 54.17%
통계량: 5,773 통계 비율 3.48%
irelia vs Garen 상대승률 : 48.16%
통계량: 5,224 통계 비율 3.15%
irelia vs Jayce 상대승률 : 56.68%
통계량: 4,522 통계 비율 2.72%
irelia vs Darius 상대승률 : 52.97%
통계량: 4,489 통계 비율 2.70%
irelia vs Riven 상대승률 : 48.95%
통계량: 4,202 통계 비율 2.53%
irelia vs Akali 상대승률 : 56.17%
통계량: 4,011 통계 비율 2.42%
irelia vs Wukong 상대승률 : 48.17%
통계량: 3,471 통계 비율 2.09%
irelia vs Shen 상대승률 : 49%
통계량: 3,284 통계 비율 1.98%
irelia vs Viego 상대승률 : 53.18%
통계량: 3,189 통계 비율 1.92%
irelia vs Ornn 상대승률 : 50.25%
통계량: 3,146 통계 비율 1.89%
irelia vs Volibear 상대승률 : 51.49%
통계량: 3,051